In [3]:
import numpy as np

$$
Supply =\left[\begin{array}{c}500\\700\\800\end{array}\right]
Demand =\left[\begin{array}{c}400\\900\\200\\500\end{array}\right]
$$


In [90]:
Supply=[500,700,800]
Cost=[[12,13,4,6],[6,4,10,11],[10,9,12,4]]
Demand=[-400,-900,-200,-500]
m=len(Supply)
n=len(Demand)
solution=np.zeros((m,n))

def basic_sloution(Supply,Cost,Demand,solution):
    i,j=0,0
    is_all_zero=False
    while is_all_zero is False:
        rest=Supply[j]+Demand[i]
        if rest>=0:
            solution[j][i]=-Demand[i]
            Supply[j]=rest
            Demand[i]=0
            i=i+1
        else:
            solution[j][i]=Supply[j]
            Demand[i]=rest
            Supply[j]=0
            j=j+1
        if all(x == 0 for x in Demand):
            is_all_zero=True
    print(Supply)
    print(Demand)
    print(Cost)
    print(solution)

basic_sloution(Supply,Cost,Demand,solution)

[0, 0, 0]
[0, 0, 0, 0]
[[12, 13, 4, 6], [6, 4, 10, 11], [10, 9, 12, 4]]
[[400. 100.   0.   0.]
 [  0. 700.   0.   0.]
 [  0. 100. 200. 500.]]


$$
BasicSolution =
\left[\begin{array}{c}
400 & 100 & 0 & 0\\
0 & 700 & 0 & 0 \\
0 & 100 & 200 & 500\end{array}\right]
$$

In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun May 24 18:05:17 2020

@author: santapo
"""

import numpy as np

supply = [500,700,800]
demand = [400,900,200,500]
costs = [[12,13,4,6],
         [6,4,10,11],
         [10,9,12,4]]



def north_west_corner(supply, demand):
    supply_copy = supply.copy()
    demand_copy = demand.copy()
    i = 0
    j = 0
    bfs = []
    while len(bfs) < len(supply) + len(demand) - 1:
        s = supply_copy[i]
        d = demand_copy[j]
        v = min(s,d)
        supply_copy[i] -= v
        demand_copy[j] -= v
        bfs.append(((i,j),v))
        if supply_copy[i] ==0 and i < len(supply) - 1:
            i += 1
        elif demand_copy[j] == 0 and j < len(demand) - 1:
            j += 1
    return bfs


def get_balenced_tp(supply,demand,costs,penalties=None):
    total_supply = sum(supply)
    total_demand = sum(demand)
    
    if total_supply < total_demand:
        if penalties is None:
            raise Exception('Supply less than demand, penlties required')
            new_supply = supply + [total_demand - total_supply]
            new_costs = costs + [penalties]
            return new_supply, demand, new_costs
    if total_supply > total_demand:
        new_demand = demand +[total_supply - total_demand]
        new_costs = costs + [[0 for _ in demand]]
        return supply, new_demand, new_costs
    return supply, demand, costs

# get u and v
def get_us_and_vs(bfs, costs):
    us = [None] * len(costs)
    vs = [None] * len(costs[0])
    us[0] = 0
    bfs_copy = bfs.copy()
    while len(bfs_copy) > 0:
        for index, bv in enumerate(bfs_copy):
            i, j = bv[0]
            if us[i] is None and vs[j] is None: continue
            
            cost = costs[i][j]
            if us[i] is None:
                us[i] = cost - vs[j]
            else:
                vs[j] = cost - us[i]
            bfs_copy.pop(index)
            break
        
    return us, vs

# calculate delta
def get_delta(bfs, costs, us, vs):
    delta = []
    for i, row in enumerate(costs):
        for j, cost in enumerate(row):
            non_basic = all([p[0] != i or p[1] != j for p, v in bfs])
            if non_basic:
                delta.append(((i,j), us[i] + vs[j] - cost))
                
    return delta

# check optimal solution
def can_be_improved(delta):
    for p, v in delta:
        if v > 0: return True
    return False


# find the largest value of delta
def get_entering_variable_position(delta):
    delta_copy = delta.copy()
    delta_copy.sort(key=lambda w: w[1])
    return delta_copy[-1][0]

# find aceptable circle in transportation problem
# return the circle and new basic variables
def get_possible_next_nodes(loop, not_visited):
    last_node = loop[-1]
    nodes_in_row = [n for n in not_visited if n[0] == last_node[0]]
    nodes_in_column = [n for n in not_visited if n[1] == last_node[1]]
    if len(loop) < 2:
        return nodes_in_row + nodes_in_column
    else:
        prev_node = loop[-2]
        row_move = prev_node[0] == last_node[0]
        if row_move: return nodes_in_column
        return nodes_in_row

def get_loop(bv_positions, ev_positions):
    def inner(loop):
        if len(loop) > 3:
            can_be_closed = len(get_possible_next_nodes(loop, [ev_positions])) == 1
            if can_be_closed: return loop
            
        not_visited = list(set(bv_positions) - set(loop))
        possible_next_nodes = get_possible_next_nodes(loop, not_visited)
        for next_node in possible_next_nodes:
            new_loop = inner(loop + [next_node])
            if new_loop: return new_loop
    
    return inner([ev_positions])

def loop_pivoting(bfs, loop):
    even_cells = loop[0::2]
    odd_cells = loop[1::2]
    get_bv = lambda pos: next(v for p, v in bfs if p == pos)
    leaving_position = sorted(odd_cells, key=get_bv)[0]
    leaving_value = get_bv(leaving_position)
    
    new_bfs = []
    for p, v in [bv for bv in bfs if bv[0] != leaving_position] + [(loop[0], 0)]:
        if p in even_cells:
            v += leaving_value
        elif p in odd_cells:
            v -= leaving_value
        new_bfs.append((p, v))
        
    return new_bfs


# final algorithm
def transportation_simplex_method(supply, demand, costs, penalties = None):
    balanced_supply, balanced_demand, balanced_costs = get_balenced_tp(supply, demand, costs)
    def inner(bfs):
        us, vs = get_us_and_vs(bfs, balanced_costs)
        delta = get_delta(bfs, balanced_costs, us, vs)
        if can_be_improved(delta):
            ev_position = get_entering_variable_position(delta)
            loop = get_loop([p for p, v in bfs], ev_position)
            return inner(loop_pivoting(bfs, loop))
        return bfs
    basic_variables = inner(north_west_corner(balanced_supply, balanced_demand))
    solution = np.zeros((len(costs), len(costs[0])))
    for (i,j), v in basic_variables:
        solution[i][j] = v
    return solution

print(transportation_simplex_method(supply, demand, costs, penalties = None))

[[300.   0. 200.   0.]
 [  0. 700.   0.   0.]
 [100. 200.   0. 500.]]


$$
OptimalSolution =
\left[\begin{array}{c}
300 & 0 & 200 & 0\\
0 & 700 & 0 & 0 \\
100 & 200 & 0 & 500\end{array}\right]
$$